<a href="https://colab.research.google.com/github/zakarka2006/famcs2024/blob/main/famcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установите библиотеки и скачайте картинки, затем по очереди запускайте блоки с кодом.

In [ ]:
!npm install -g degit
!pip install opencv-python
!pip install easyocr


added 1 package, and audited 2 packages in 3s

found 0 vulnerabilities
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.meta

In [ ]:
!npx degit zakarka2006/famcs2024/images -f ./images

> cloned zakarka2006/famcs2024#HEAD to ./images


# Letters 1
Начнем с букв: почему-то я сразу начал думать в сторону opencv, как минимум для решения пунктов 2-3, поэтому решение получилось немного "костыльным". Я вырезал отдельные картинки букв размером 60х60 пикселей, но перед этим выкрутил контраст на максимум, чтобы не терять деталей. (Все картинки в папке `./images`)

Как я считаю буквы:
1. Преобразование картинки: функция переводит картинку в ч/б формат и увеличивает контрастность
2. Предобработка изображения: функция загружает изображение и преобразует его в черно-белое. Затем применяется бинаризация с инверсией, чтобы буквы стали белыми на черном фоне.
3. Поиск контуров: на этом этапе я нахожу контуры на бинарном изображении. Найденные контура - это области, содержащие буквы.
4. Подготовка шаблонов для букв: функция создает шаблоны для каждой буквы. Изображения букв предварительно обрабатываются и извлекаются их контуры. Шаблоны сохраняются в словаре для последующего использования.
5. Сопоставление контуров: каждый найденный контур на картинке сопоставляется с шаблоном буквы, в итоге для каждого контура выбирается более подходящий шаблон буквы

Конечно, я пробовал прикрутить ocr для этой задачи, но не все буквы определялись правильно со 100% вероятностью. Поэтому отошел от этой идеи. Работу этого кода проверял вручную на небольших картинках по 8х8 букв, ошибок не нашел, так что должно работать :D.

In [ ]:
import cv2
import numpy as np
from collections import Counter
from PIL import Image, ImageEnhance

Image.MAX_IMAGE_PIXELS = 100000000
def maximize_contrast(image_path):
    image = Image.open(image_path)
    image = image.convert("L")
    enhancer = ImageEnhance.Contrast(image)
    max_contrast_image = enhancer.enhance(10)
    max_contrast_image.save(f"{image_path[:-4]}_contrast.png")

def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def create_letter_templates():
    templates = {}
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        template_path = f"./images/letters/{letter}.png"
        template_image = preprocess_image(template_path)
        contours = find_contours(template_image)
        if contours:
            templates[letter] = contours
    return templates

def match_letters(image, templates):
    detected_letters = []
    contours = find_contours(image)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        letter_image = image[y:y+h, x:x+w]
        max_match = float("inf")
        matched_letter = ""
        for letter, template_contours in templates.items():
            for template_contour in template_contours:
                match = cv2.matchShapes(template_contour, contour, cv2.CONTOURS_MATCH_I3, 0.0)
                if match < max_match:
                    max_match = match
                    matched_letter = letter
        detected_letters.append(matched_letter)
    return detected_letters

def count_letters():
    maximize_contrast("./images/letters.png")
    binary_image = preprocess_image("./images/letters_contrast.png")
    templates = create_letter_templates()
    detected_letters = match_letters(binary_image, templates)

    letter_count = Counter(detected_letters)
    sorted_letters = sorted(letter_count.items())
    total_count = 0

    for letter, count in sorted_letters:
        if not letter.isalpha():
            continue
        total_count += count
        print(f"{letter}: {count}")

    print(f"Total: {total_count}")

count_letters()

A: 651
B: 565
C: 935
D: 498
E: 20
F: 183
G: 557
H: 740
I: 797
J: 322
K: 562
L: 188
M: 727
N: 762
O: 371
P: 86
Q: 188
R: 657
S: 630
T: 169
U: 763
V: 888
W: 526
X: 70
Y: 857
Z: 744
Total: 13456


# Letters 2
Самый простой пункт, по моему мнению, т.к. все просто сводится к нахождению контуров букв и отрисовки рамок вокруг них.

Итоговая картинка лежит в `./images/letters_rectangles.png`



In [ ]:
import cv2
import numpy as np
import time

def add_bounding_boxes_letters():
    padding = 2
    thickness = 3
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    kernel = np.ones((3, 3), np.uint8)
    img_erode = cv2.erode(thresh, kernel, iterations=1)

    contours, _ = cv2.findContours(img_erode, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output = cv2.imread("./images/letters.png")
    padding += thickness
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0 # для букв возле левой границы
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(output, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(output, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(output, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(output, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

    cv2.imwrite("./images/letters_rectangles.png", output)
    print("Done")

add_bounding_boxes_letters()

Done


# Letters 3
Можно немного изменить второй код и вместо рамок рисовать фон под буквами
Как работает:
1. Поиск контуров
2. Выбор самого встречающегося цвета в пределах контура
3. Наложение прямоугольной рамки
4. Наложение буквы на итоговую картинку с помощью маски

Итоговая картинка лежит в `./images/letters_backgrounds.png`


In [ ]:
import cv2
import numpy as np
import time

def add_background_letters():
    maximize_contrast("./images/letters.png")
    img = cv2.imread("./images/letters_contrast.png")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    margin = 3

    imgc = cv2.imread("./images/letters.png")
    output = imgc.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        letter_color = np.mean(output[y:y+h, x:x+w], axis=(0, 1))

        background_color = [255 - int(c) for c in letter_color]

        cv2.rectangle(output, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(output[y:y+h, x:x+w], output[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        output[y:y+h, x:x+w] = cv2.add(bg, fg)

    cv2.imwrite("./images/letters_backgrounds.png", output)
    print("Done")

add_background_letters()

Done


# Words 5, 6
Так как мое "чудесное" решение зависит от нахождения контуров, то пришлось придумывать что-то другое для слов, потому что они расположены хаотично на картинке. Ввиду недостатка времени пришлось костылять: сначала пробовал искать контуры для слов только с помощью OpenCV, но еще во время поиска инстументов для распознавания символов наткнулся на библиотеку `easyocr`, так что первоначальное определение контуров она взяла на себя, а затем я с помощью OpenCV доделывал работу. Идея такая:
1. Получить список контуров с помощью `easyocr`
2. Отсортировать слова на 2 типа: вертикальные и горизонтальные
3. Внутри этих контуров уже искать настоящие контуры слов
4. Финальные манипуляции с картинкой

Разделить и почистить код времени не было, так что тут все 3 пункта в одной куче.

Итоговые картинки: `./images/words_rectangles.png`, `./images/words_backgrounds.png`

P.S.: идеального результата добиться не удалось, опять же, из-за хаотичного расположения слов, но по бОльшая часть слов определяется хорошо(если контур нашелся, то с рамкой и фоном все ок будет)

P.S. 2: а еще есть косяк с разделением слова на 2 части, т.е. есть линия посреди слова :(

In [ ]:
import easyocr
import cv2
import random
import PIL

reader = easyocr.Reader(["en"])

In [192]:
# Выполняется около 3-х минут
image_path = "./images/words.png"
maximize_contrast(image_path)
image = cv2.imread("./images/words_contrast.png")

bounds = reader.detect(image_path, mag_ratio=0.7, add_margin=0)

In [193]:
def draw_boxes(image, bounds, width=2):
    horizontal_bounds = []
    vertical_bounds = []
    for bound in bounds[0][0]:
        x_min, x_max, y_min, y_max = bound
        top_left = (x_min, y_min)
        bottom_right = (x_max, y_max)
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        if (y_max-y_min > x_max-x_min):
            vertical_bounds.append(bound)
            # vert
            color = (0, 255, 0)
        else:
            horizontal_bounds.append(bound)
            # hor
            color = (0, 0, 255)
        cv2.rectangle(image, top_left, bottom_right, color, width)
    horizontal_bounds = sorted(horizontal_bounds, key=lambda x: (x[1], x[0]))
    vertical_bounds = sorted(vertical_bounds, key=lambda x: (x[1], x[0]))
    return image, horizontal_bounds, vertical_bounds

image_with_boxes, horizontal_bounds, vertical_bounds = draw_boxes(image, bounds)
image = cv2.imread(image_path)
final = ""

def find_words(img, x_min, y_min, hor=True):
    global final
    words = []

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (12, 3))

    dilation = cv2.dilate(thresh, rect_kernel, iterations=1)

    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (hor):
            words.append((x_min + x, y_min + y, w, h))
        else:
            words.append((x_min + y, y_max - x - w, h, w))

    return words

words_hor = []
words_vert = []
crop = []
for bound in horizontal_bounds:
    x_min, x_max, y_min, y_max = bound
    crop = image[y_min:y_max, x_min:x_max]
    words_hor.extend(find_words(crop, x_min, y_min))

for bound in vertical_bounds:
    x_min, x_max, y_min, y_max = bound
    crop = cv2.rotate(image[y_min:y_max, x_min:x_max], cv2.ROTATE_90_CLOCKWISE)
    final = crop
    words_vert.extend(find_words(crop, x_min, y_min, False))

def draw_rectangles_words():
    thickness = 2
    padding = 4
    img = cv2.imread(image_path)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    margin = 3
    imgc = img.copy()
    background_img = img.copy()
    for word in words_hor:
        x, y, w, h = word
        if (w < 80 or h > 63 or h < 30):
            continue
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(img, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(img, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(img, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(img, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

        letter_color = np.mean(background_img[y:y+h, x:x+w], axis=(0, 1))
        background_color = [255 - int(c) for c in letter_color]
        cv2.rectangle(background_img, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(background_img[y:y+h, x:x+w], background_img[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        background_img[y:y+h, x:x+w] = cv2.add(bg, fg)

    for word in words_vert:
        x, y, w, h = word
        if (h < 80 or w > 63 or w < 30):
            continue
        x_pad = max(x - padding - 1, 0)
        y_pad = max(y - padding - 1, 0)
        dx = (0 - (x - padding - 1)) if x_pad == 0 else 0
        w_pad = w + 2*padding + 1 - dx
        h_pad = h + 2*padding + 1

        border_color = (0, 0, 0)
        cv2.line(img, (x_pad, y_pad), (x_pad + w_pad, y_pad), border_color, thickness)
        cv2.line(img, (x_pad, y_pad + h_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        cv2.line(img, (x_pad + w_pad, y_pad), (x_pad + w_pad, y_pad + h_pad), border_color, thickness)
        if dx == 0:
            cv2.line(img, (x_pad, y_pad), (x_pad, y_pad + h_pad), border_color, thickness)

        letter_color = np.mean(background_img[y:y+h, x:x+w], axis=(0, 1))
        background_color = [255 - int(c) for c in letter_color]
        cv2.rectangle(background_img, (x-margin, y-margin), (x+w+margin, y+h+margin), background_color, -1)

        letter_region = imgc[y:y+h, x:x+w]
        mask = binary[y:y+h, x:x+w]
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(background_img[y:y+h, x:x+w], background_img[y:y+h, x:x+w], mask=mask_inv)
        fg = cv2.bitwise_and(letter_region, letter_region, mask=mask)
        background_img[y:y+h, x:x+w] = cv2.add(bg, fg)

    return img, background_img

rect_img, background_img = draw_rectangles_words()
cv2.imwrite("./images/words_rectangles.png", rect_img)
cv2.imwrite("./images/words_backgrounds.png", background_img)

[[0, 187, 9289, 9359], [39, 226, 460, 531], [31, 250, 5484, 5531], [46, 265, 3562, 3617], [218, 265, 9789, 9835], [140, 281, 2257, 2320], [54, 289, 992, 1109], [0, 312, 3625, 3695], [7, 312, 6648, 6710], [54, 312, 4554, 4609], [210, 312, 7585, 7656], [31, 335, 1148, 1203], [109, 335, 1585, 1656], [31, 343, 23, 93], [31, 343, 8476, 8578], [39, 351, 609, 671], [39, 382, 1250, 1539], [109, 382, 3742, 3812], [171, 398, 750, 882], [164, 406, 7320, 7500], [179, 406, 9914, 9968], [203, 414, 8609, 8664], [187, 445, 3890, 3945], [242, 460, 6781, 6835], [335, 468, 1000, 1062], [62, 476, 2429, 2546], [132, 476, 8671, 8804], [78, 484, 890, 968], [312, 492, 468, 523], [320, 492, 2875, 2929], [304, 531, 2570, 2625], [328, 546, 2718, 2773], [148, 562, 5507, 5695], [109, 570, 6539, 6648], [265, 570, 6382, 6515], [273, 585, 2968, 3031], [156, 601, 5679, 5828], [187, 601, 4000, 4132], [31, 609, 8304, 8445], [210, 609, 3054, 3171], [242, 609, 8812, 8953], [265, 609, 265, 406], [328, 632, 4187, 4242], [37

True